<a href="https://colab.research.google.com/github/jtuckayo/scratchpad/blob/main/DuckDB_%26_Iceberg_The_Future_of_Lightweight_Data_Management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# prompt: install duckdb and sqlite in this notebook

!pip install duckdb
!pip install pysqlite3


In [6]:
import duckdb

In [7]:
pytable = duckdb.sql("SELECT * FROM 's3://us-prd-motherduck-open-datasets/who_ambient_air_quality/csv/who_ambient_air_quality_database_version_v6_april_2023.csv'").arrow()

In [ ]:
pytable

In [8]:
# prompt: install pyiceberg

!pip install pyiceberg


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 822.1/822.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 12.1 MB/s eta 0:00:00


In [ ]:
!mkdir "/content/warehouse"

In [ ]:
from pyiceberg.catalog.sql import SqlCatalog

warehouse_path = "/content/warehouse"
catalog = SqlCatalog(
    "default",
    **{
      "uri":f"sqlite:///{warehouse_path}/pyiceberg_catalog_db",
      "warehouse":f"file://{warehouse_path}",
    },
)

In [ ]:
catalog.create_namespace("default")

table = catalog.create_table(
    "default.air_quality",
    schema = pytable.schema,
)

In [ ]:
table.append(pytable)
len(table.scan().to_arrow())

In [ ]:
con_ice = table.scan().to_duckdb(table_name = "air")

In [ ]:
con_ice.sql('FROM air;')

In [ ]:
df = table.scan(row_filter="city == 'Berlin'").to_arrow()

In [ ]:
table.overwrite(df)

In [ ]:
con_ice = table.scan().to_duckdb(table_name = "air")

In [ ]:
con_ice.sql('FROM air;')